## LISS4 Image Processing using XArray and Dask

This notebook shows how to pre-process scenes from the LISS4 sensor aboard ResourceSat2/2A. 

The tutorial covers the steps required to download the imagery and apply the pre-processing steps required to make this data ready for analysis - specifically how to programmatically convert the DN values to TOA Reflectance. See [this article](https://spatialthoughts.com/2023/12/25/liss4-processing-xarray/) for a detailed guide.


### Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment. If you are running this on your own system, you may install the following packages via `conda` and `pip`.

```
conda create --name liss4
conda activate liss4
conda install -c conda-forge rioxarray dask jupyterlab -y
pip install ephem
```

In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
  !pip install rioxarray
  !pip install ephem

In [ ]:
import datetime
import ephem
import math
import os
import rioxarray as rxr
import xarray as xr
import zipfile

Users should download their data from the [Bhoonidhi portal](https://bhoonidhi.nrsc.gov.in/bhoonidhi/index.html). Replace the `liss4_zip` value with your own downloaded file. 

In [ ]:
def download(url):
    filename = os.path.basename(url)
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

liss4_zip = 'RAF20FEB2023032197010000064SSANSTUC00GTDC.zip'
data_url = 'https://storage.googleapis.com/spatialthoughts-public-data/'

download(data_url + 'liss4/' + liss4_zip)

Next we initiate a local dask cluster. This uses all the available cores on your machine in parallel. You will see a link to the Dask Dashboard. 

In [ ]:
from dask.distributed import Client, progress
client = Client()  # set up local cluster on the machine
client

### Data Pre-Processing


We first unzip the scene zip file to a folder.

In [ ]:
with zipfile.ZipFile(liss4_zip) as zf:
  # The LISS4 zip files contain a folder with all the data
  # Get the folder name
  foldername = [info.filename for info in zf.infolist() if info.is_dir()][0]
  # Extract all the data
  zf.extractall()

print(f'Extracted the files to {foldername}.')

We read the metadata and extract the relevant informaiton. The metadata is contained in the text file `BAND_META.txt`. The format simple key-value pairs of the form `ProductID= 2326285231`. We parse the text into a dictionary.

In [ ]:
metadata_filename = 'BAND_META.txt'
metadata_filepath = os.path.join(foldername, metadata_filename)

metadata = {}
with open(metadata_filepath) as f:
  for line in f:
    key, value = line.split('=')
    metadata[key.strip()] = value.strip()

scene_id = metadata['OTSProductID']
print(f'Metadata extracted successfully for scene: {scene_id}')

LISS4 images come as 3 individual GeoTIFF rasters for each band. The image files are named `BAND2.tif`, `BAND3.tif` and `BAND4.tif` . We read them using `rioxarray`.

In [ ]:
b2_path = os.path.join(foldername, 'BAND2.tif')
b3_path = os.path.join(foldername, 'BAND3.tif')
b4_path = os.path.join(foldername, 'BAND4.tif')

b2_ds = rxr.open_rasterio(b2_path, chunks=True)
b3_ds = rxr.open_rasterio(b3_path, chunks=True)
b4_ds = rxr.open_rasterio(b4_path, chunks=True)

Create a XArray Dataset by stacking individual band images. The scene has a NoData value of 0. So we set the correct NoData value before further processing.

In [ ]:
scene = xr.concat([b2_ds, b3_ds, b4_ds], dim='band').assign_coords(
    band=['BAND2', 'BAND3', 'BAND4'])

scene = scene.where(scene != 0)
scene.name = scene_id
scene


## Converting DNs to Reflectances

The pixel values of the source images are DN values that need to be converted to reflectances before they can be used for analysis.

The correction formulae and sensor parameters are published in the following paper

*Sharma, Anu & Badarinath, K. & Roy, Parth. (2008). Corrections for atmospheric and adjacency effects on high resolution sensor data a case study using IRS-P6 LISS-IV data. https://www.isprs.org/proceedings/xxxvii/congress/8_pdf/3_wg-viii-3/05.pdf*

In [ ]:
acq_date_str = metadata['DateOfPass']
# Date is in the format 04-MAR-2023
acq_date = datetime.datetime.strptime(acq_date_str, '%d-%b-%Y')

sun_elevation_angle = metadata['SunElevationAtCenter']
sun_zenith_angle = 90 - float(sun_elevation_angle)

We need to compute the Earth-Sun distance at the date of acquisition. We use the `pyephm` library to obtain this.

In [ ]:
observer = ephem.Observer()
observer.date = acq_date
sun = ephem.Sun()
sun.compute(observer)
d = sun.earth_distance

Define the Saturation Radiance for each band. These come from the [RESOURCESAT-2 Data Users’ Handbook ](https://www.euromap.de/download/R2_data_user_handbook.pdf) and are available in metadata as well.

In [ ]:
b2_sr = 47.5614
b3_sr = 45.6990
b4_sr = 31.5000

Define ex-atmospheric solar irradiance values for each band ESUN for each band as published in [*At-sensor Solar Exo-atmospheric Irradiance, Rayleigh Optical
Thickness and Spectral parameters of RS-2 Sensors*](https://www.researchgate.net/profile/Senthil-Kumar-135/post/What-is-the-formula-for-converting-DN-values-to-reflectance-value-for-IRS-R2-LISS-IV-image/attachment/59d61ff4c49f478072e97d4b/AS%3A271753849311232%401441802574228/download/RS2-Spectral+Characteristics.pdf).

In [ ]:
b2_esun = 181.89
b3_esun = 156.96
b4_esun = 110.48

Define other contants needed for computation.

In [ ]:
pi = math.pi
sun_zenith_angle_rad = math.radians(sun_zenith_angle)

Convert DN to Radiance.

In [ ]:
b2_dn = scene.sel(band='BAND2')
b3_dn = scene.sel(band='BAND3')
b4_dn = scene.sel(band='BAND4')

In [ ]:
b2_rad = b2_dn*b2_sr/1024
b3_rad = b3_dn*b3_sr/1024
b4_rad = b4_dn*b4_sr/1024

Convert Radiance to TOA Reflectance.

In [ ]:
b2_ref = (pi*b2_rad*d*d)/(b2_esun*math.cos(sun_zenith_angle_rad))
b3_ref = (pi*b3_rad*d*d)/(b3_esun*math.cos(sun_zenith_angle_rad))
b4_ref = (pi*b4_rad*d*d)/(b4_esun*math.cos(sun_zenith_angle_rad))

Stack the bands into a single XArray Dataset.

In [21]:
reflectance_bands = [b2_ref, b3_ref, b4_ref]
scene_ref = xr.concat(reflectance_bands, dim='band').assign_coords(
    band=['BAND2', 'BAND3', 'BAND4']).chunk('auto')
scene_ref.name = scene_id
scene_ref

<xarray.DataArray 'RAF20FEB2023032197010000064SSANSTUC00GTDC' (band: 3,
                                                               y: 16615,
                                                               x: 17733)>
dask.array<rechunk-merge, shape=(3, 16615, 17733), dtype=float32, chunksize=(1, 2674, 12547), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 7.15e+05 7.15e+05 7.15e+05 ... 8.036e+05 8.036e+05
  * y            (y) float64 1.468e+06 1.468e+06 ... 1.385e+06 1.385e+06
    spatial_ref  int64 0
  * band         (band) <U5 'BAND2' 'BAND3' 'BAND4'

### Write the Results to Disk

Our DataArray is structured to have 'band' as a dimension which makes it easy for data manipulation and processing. But for use in standard GIS software - it is better to create an XArray Dataset with each band as a variable.

In [ ]:
output_ds = scene_ref.to_dataset('band')
output_ds

Define the options for the output file. We use the COG driver to create a [Cloud-Optimized GeoTIFF](https://www.cogeo.org/) file. 

In [ ]:
output_file = f'{scene_id}.tif'

output_options = {
    'driver': 'COG',
    'compress': 'deflate',
    'num_threads': 'all_cpus',
    'windowed': False # set True if you run out of RAM
}

Write the raster. Dask will now run the processing graph on the local cluster.

In [ ]:
output_ds[['BAND2', 'BAND3', 'BAND4']].rio.to_raster(
    output_file, **output_options)
print(f'Output file created {output_file}')